In [1]:
import pandas as pd


In [2]:
data = pd.read_csv("verified_online (1).csv")
data.head()

,phish_id,url,phish_detail_url,submission_time,verified,verification_time,online,target
0,6975070,https://smcc-comm.com/,http://www.phishtank.com/phish_detail.php?phis...,2021-02-19T01:55:20+00:00,yes,2021-02-19T01:58:51+00:00,yes,Other
1,6975069,https://payattentiontotheinput.storenewrakuten...,http://www.phishtank.com/phish_detail.php?phis...,2021-02-19T01:47:35+00:00,yes,2021-02-19T01:49:05+00:00,yes,Other
2,6975066,https://3csystem.vn/wp-config/mua,http://www.phishtank.com/phish_detail.php?phis...,2021-02-19T01:22:50+00:00,yes,2021-02-19T01:27:29+00:00,yes,Other
3,6975065,http://3csystem.vn/wp-config/mua,http://www.phishtank.com/phish_detail.php?phis...,2021-02-19T01:22:36+00:00,yes,2021-02-19T01:27:29+00:00,yes,Other
4,6975064,http://3csystem.vn/wp-config/mua/,http://www.phishtank.com/phish_detail.php?phis...,2021-02-19T01:22:29+00:00,yes,2021-02-19T01:27:29+00:00,yes,Other


In [3]:
data.shape
data.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10898 entries, 0 to 10897
Data columns (total 8 columns):
phish_id             10898 non-null int64
url                  10898 non-null object
phish_detail_url     10898 non-null object
submission_time      10898 non-null object
verified             10898 non-null object
verification_time    10898 non-null object
online               10898 non-null object
target               10898 non-null object
dtypes: int64(1), object(7)
memory usage: 681.2+ KB


A dataset containing 5500 URLs will be considered out of this datset as the source is updated frequently and could lead to data imbalance

In [7]:
# selecting 5500 samples from phishing datset
phishurl = data.sample(n = 5500, random_state = 14).copy()
phishurl = phishurl.reset_index(drop=True)
phishurl.head()

,phish_id,url,phish_detail_url,submission_time,verified,verification_time,online,target
0,6420790,https://icipedudu-my.sharepoint.com/:x:/r/pers...,http://www.phishtank.com/phish_detail.php?phis...,2020-02-26T20:32:35+00:00,yes,2020-03-09T07:15:03+00:00,yes,Other
1,6941435,https://ebay.portlibertecontractor.com/urXkq/X...,http://www.phishtank.com/phish_detail.php?phis...,2021-01-28T16:19:55+00:00,yes,2021-02-01T02:09:24+00:00,yes,"eBay, Inc."
2,6352299,https://eliotecae-my.sharepoint.com/personal/s...,http://www.phishtank.com/phish_detail.php?phis...,2020-01-10T14:06:20+00:00,yes,2020-01-13T23:39:18+00:00,yes,Other
3,6968112,http://paypal-uk-webcmd-login.done-login-acces...,http://www.phishtank.com/phish_detail.php?phis...,2021-02-14T14:02:06+00:00,yes,2021-02-14T14:04:10+00:00,yes,Other
4,6974084,https://deliverry-safe.biz/ipko1613578817628824,http://www.phishtank.com/phish_detail.php?phis...,2021-02-18T10:36:37+00:00,yes,2021-02-18T10:37:48+00:00,yes,PKO Polish Bank


In [8]:
phishurl.shape

(5500, 8)

In [20]:
data1 = pd.read_csv("Benign_list_big_final.csv") #this dataset was obtained form the Canadian institue of Cybersecurity
data1.columns = ['URLs']
data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35377 entries, 0 to 35376
Data columns (total 1 columns):
URLs    35377 non-null object
dtypes: object(1)
memory usage: 276.5+ KB


In [7]:
legiurl = data1.sample(n = 5500, random_state = 14).copy()
legiurl = legiurl.reset_index(drop=True)
legiurl.head()

,URLs
0,http://censor.net.ua/video_news/335293/epopeya...
1,https://paytm.com/offer/blog/faq/got-a-better-...
2,http://sfglobe.com/2015/02/23/american-sniper-...
3,https://twitter.com/home?status=%E3%83%8C%E3%8...
4,http://torrentdn.com/bbs/s.php?bo_table=torren...


Feature Extraction:

In this step, features are extracted from the URLs dataset.

The extracted features are categorized into

Address Bar based Features

Address Bar Based Features:

Many features can be extracted that can be consided as address bar base features. Out of them, below mentioned were considered for this project.

Domain of URL

IP Address in URL

"@" Symbol in URL

Length of URL

Depth of URL

Redirection "//" in URL

"http/https" in Domain name

Using URL Shortening Services “TinyURL”

Prefix or Suffix "-" in Domain

In [36]:
from urllib.parse import urlparse,urlencode
import ipaddress
import re


3.1.1. Domain of the URL
Here, we are just extracting the domain present in the URL. This feature doesn't have much significance in the training. May even be dropped while training the model.

In [37]:
def getDomain(url):  
  domain = urlparse(url).netloc
  if re.match(r"^www.",domain):
       domain = domain.replace("www.","")
  return domain

 IP Address in the URL
Checks for the presence of IP address in the URL. URLs may have IP address instead of domain name. If an IP address is used as an alternative of the domain name in the URL, we can be sure that someone is trying to steal personal information with this URL.

If the domain part of URL has IP address, the value assigned to this feature is 1 (phishing) or else 0 (legitimate).

In [38]:
def havingIP(url):
  try:
    ipaddress.ip_address(url)
    ip = 1
  except:
    ip = 0
  return ip

"@" Symbol in URL
Checks for the presence of '@' symbol in the URL. Using “@” symbol in the URL leads the browser to ignore everything preceding the “@” symbol and the real address often follows the “@” symbol.

If the URL has '@' symbol, the value assigned to this feature is 1 (phishing) or else 0 (legitimate).

In [39]:
def haveAtSign(url):
  if "@" in url:
    at = 1    
  else:
    at = 0    
  return at

Length of URL
Computes the length of the URL. Phishers can use long URL to hide the doubtful part in the address bar. In this project, if the length of the URL is greater than or equal 54 characters then the URL classified as phishing otherwise legitimate.

If the length of URL >= 54 , the value assigned to this feature is 1 (phishing) or else 0 (legitimate).

In [40]:
def getLength(url):
  if len(url) < 54:
    length = 0            
  else:
    length = 1            
  return length

Depth of URL
Computes the depth of the URL. This feature calculates the number of sub pages in the given url based on the '/'.

The value of feature is a numerical based on the URL.

In [41]:
def getDepth(url):
  s = urlparse(url).path.split('/')
  depth = 0
  for j in range(len(s)):
    if len(s[j]) != 0:
      depth = depth+1
  return depth

Redirection "//" in URL
Checks the presence of "//" in the URL. The existence of “//” within the URL path means that the user will be redirected to another website. The location of the “//” in URL is computed. We find that if the URL starts with “HTTP”, that means the “//” should appear in the sixth position. However, if the URL employs “HTTPS” then the “//” should appear in seventh position.

If the "//" is anywhere in the URL apart from after the protocal, thee value assigned to this feature is 1 (phishing) or else 0 (legitimate).

In [42]:
def redirection(url):
  pos = url.rfind('//')
  if pos > 6:
    if pos > 7:
      return 1
    else:
      return 0
  else:
    return 0

 Using URL Shortening Services “TinyURL”
URL shortening is a method on the “World Wide Web” in which a URL may be made considerably smaller in length and still lead to the required webpage. This is accomplished by means of an “HTTP Redirect” on a domain name that is short, which links to the webpage that has a long URL.

If the URL is using Shortening Services, the value assigned to this feature is 1 (phishing) or else 0 (legitimate).

In [43]:

# 7.Existence of “HTTPS” Token in the Domain Part of the URL (https_Domain)
def httpDomain(url):
  domain = urlparse(url).netloc
  if 'https' in domain:
    return 1
  else:
    return 0

 Using URL Shortening Services “TinyURL”
URL shortening is a method on the “World Wide Web” in which a URL may be made considerably smaller in length and still lead to the required webpage. This is accomplished by means of an “HTTP Redirect” on a domain name that is short, which links to the webpage that has a long URL.

If the URL is using Shortening Services, the value assigned to this feature is 1 (phishing) or else 0 (legitimate).

In [44]:
shortening_services = r"bit\.ly|goo\.gl|shorte\.st|go2l\.ink|x\.co|ow\.ly|t\.co|tinyurl|tr\.im|is\.gd|cli\.gs|" \
                      r"yfrog\.com|migre\.me|ff\.im|tiny\.cc|url4\.eu|twit\.ac|su\.pr|twurl\.nl|snipurl\.com|" \
                      r"short\.to|BudURL\.com|ping\.fm|post\.ly|Just\.as|bkite\.com|snipr\.com|fic\.kr|loopt\.us|" \
                      r"doiop\.com|short\.ie|kl\.am|wp\.me|rubyurl\.com|om\.ly|to\.ly|bit\.do|t\.co|lnkd\.in|db\.tt|" \
                      r"qr\.ae|adf\.ly|goo\.gl|bitly\.com|cur\.lv|tinyurl\.com|ow\.ly|bit\.ly|ity\.im|q\.gs|is\.gd|" \
                      r"po\.st|bc\.vc|twitthis\.com|u\.to|j\.mp|buzurl\.com|cutt\.us|u\.bb|yourls\.org|x\.co|" \
                      r"prettylinkpro\.com|scrnch\.me|filoops\.info|vzturl\.com|qr\.net|1url\.com|tweez\.me|v\.gd|" \
                      r"tr\.im|link\.zip\.net"

In [45]:

# 8. Checking for Shortening Services in URL (Tiny_URL)
def tinyURL(url):
    match=re.search(shortening_services,url)
    if match:
        return 1
    else:
        return 0

 Prefix or Suffix "-" in Domain
Checking the presence of '-' in the domain part of URL. The dash symbol is rarely used in legitimate URLs. Phishers tend to add prefixes or suffixes separated by (-) to the domain name so that users feel that they are dealing with a legitimate webpage.

If the URL has '-' symbol in the domain part of the URL, the value assigned to this feature is 1 (phishing) or else 0 (legitimate).

In [46]:
def prefixSuffix(url):
    if '-' in urlparse(url).netloc:
        return 1            # phishing
    else:
        return 0            # legitimate

Web Traffic

This feature measures the popularity of the website by determining the number of visitors and the number of pages they visit. However, since phishing websites live for a short period of time, they may not be recognized by the Alexa database (Alexa the Web Information Company., 1996). By reviewing our dataset, we find that in worst scenarios, legitimate websites ranked among the top 100,000. Furthermore, if the domain has no traffic or is not recognized by the Alexa database, it is classified as “Phishing”.

If the rank of the domain < 100000, the vlaue of this feature is 1 (phishing) else 0 (legitimate).

In [47]:
# importing required packages for this section
import re
from bs4 import BeautifulSoup
import whois
import urllib
import urllib.request
from datetime import datetime

In [48]:
#Web traffic (Web_Traffic)
def web_traffic(url):
  try:
    #Filling the whitespaces in the URL if any
    url = urllib.parse.quote(url)
    rank = BeautifulSoup(urllib.request.urlopen("http://data.alexa.com/data?cli=10&dat=s&url=" + url).read(), "xml").find(
        "REACH")['RANK']
    rank = int(rank)
  except TypeError:
        return 1
  if rank <100000:
    return 1
  else:
    return 0

Age of Domain

This feature can be extracted from WHOIS database. Most phishing websites live for a short period of time. The minimum age of the legitimate domain is considered to be 12 months for this project. Age here is nothing but different between creation and expiration time.

If age of domain > 12 months, the vlaue of this feature is 1 (phishing) else 0 (legitimate).

In [49]:
#Survival time of domain: The difference between termination time and creation time (Domain_Age)  
def domainAge(domain_name):
  creation_date = domain_name.creation_date
  expiration_date = domain_name.expiration_date
  if (isinstance(creation_date,str) or isinstance(expiration_date,str)):
    try:
      creation_date = datetime.strptime(creation_date,'%Y-%m-%d')
      expiration_date = datetime.strptime(expiration_date,"%Y-%m-%d")
    except:
      return 1
  if ((expiration_date is None) or (creation_date is None)):
      return 1
  elif ((type(expiration_date) is list) or (type(creation_date) is list)):
      return 1
  else:
    ageofdomain = abs((expiration_date - creation_date).days)
    if ((ageofdomain/30) < 6):
      age = 1
    else:
      age = 0
  return age

End Period of Domain

This feature can be extracted from WHOIS database. For this feature, the remaining domain time is calculated by finding the different between expiration time & current time. The end period considered for the legitimate domain is 6 months or less for this project.

If end period of domain > 6 months, the vlaue of this feature is 1 (phishing) else 0 (legitimate).

In [50]:

# 14.End time of domain: The difference between termination time and current time (Domain_End) 
def domainEnd(domain_name):
  expiration_date = domain_name.expiration_date
  if isinstance(expiration_date,str):
    try:
      expiration_date = datetime.strptime(expiration_date,"%Y-%m-%d")
    except:
      return 1
  if (expiration_date is None):
      return 1
  elif (type(expiration_date) is list):
      return 1
  else:
    today = datetime.now()
    end = abs((expiration_date - today).days)
    if ((end/30) < 6):
      end = 0
    else:
      end = 1
  return end


3.3.1. IFrame Redirection
IFrame is an HTML tag used to display an additional webpage into one that is currently shown. Phishers can make use of the “iframe” tag and make it invisible i.e. without frame borders. In this regard, phishers make use of the “frameBorder” attribute which causes the browser to render a visual delineation.

If the iframe is empty or repsonse is not found then, the value assigned to this feature is 1 (phishing) or else 0 (legitimate).

In [97]:
def rightClick(response):
  if response == "":
    return 1
  else:
    if re.findall(r"event.button ?== ?2", response.text):
      return 0
    else:
      return 1

In [98]:
def forwarding(response):
  if response == "":
    return 1
  else:
    if len(response.history) <= 2:
      return 0
    else:
      return 1

 Computing URL Features
Create a list and a function that calls the other functions and stores all the features of the URL in the list. We will extract the features of each URL and append to this list.

In [113]:
#Function to extract features
def featureExtraction(url,label):

  features = []
  #Address bar based features (10)
  features.append(getDomain(url))
  features.append(havingIP(url))
  features.append(haveAtSign(url))
  features.append(getLength(url))
  features.append(getDepth(url))
  features.append(redirection(url))
  features.append(httpDomain(url))
  features.append(tinyURL(url))
  features.append(prefixSuffix(url))


  features.append(label)
  
  return features

In [114]:
legiurl.shape

(5500, 1)

In [115]:
#Extracting the feautres & storing them in a list
legi_features = []
label = 0

for i in range(0, 5500):
  url = legiurl['URLs'][i]
  legi_features.append(featureExtraction(url,label))


In [116]:
#converting the list to dataframe
feature_names = ['Domain', 'Have_IP', 'Have_At', 'URL_Length', 'URL_Depth','Redirection','https_Domain', 'TinyURL', 'Prefix/Suffix','Label']



In [129]:

legitimate = pd.DataFrame(legi_features, columns= feature_names)
legitimate.head(10)

,Domain,Have_IP,Have_At,URL_Length,URL_Depth,Redirection,https_Domain,TinyURL,Prefix/Suffix,Label
0,censor.net.ua,0,0,1,3,0,0,0,0,0
1,paytm.com,0,0,1,4,0,0,0,0,0
2,sfglobe.com,0,0,1,4,0,0,0,0,0
3,twitter.com,0,0,1,1,0,0,0,0,0
4,torrentdn.com,0,0,1,2,0,0,0,0,0
5,babal.net,0,0,1,4,0,0,0,0,0
6,torcache.net,0,0,1,2,0,0,0,0,0
7,udn.com,0,0,1,4,0,0,0,0,0
8,emgn.com,0,0,1,2,0,0,0,0,0
9,mirtesen.ru,0,0,1,2,0,0,0,0,0


In [130]:
legitimate.to_csv('legitimate.csv', index= False)
#storing it to a legitimate dataset called legitimate.csv

In [131]:
#Extracting the feautres & storing them in a list
phish_features = []
label = 1
for i in range(0, 5500):
  url = phishurl['url'][i]
  phish_features.append(featureExtraction(url,label))

#converting the list to dataframe
feature_names = ['Domain', 'Have_IP', 'Have_At', 'URL_Length', 'URL_Depth','Redirection', 
                      'https_Domain', 'TinyURL', 'Prefix/Suffix','Label']

phishing = pd.DataFrame(phish_features, columns= feature_names)
phishing.head(10)

,Domain,Have_IP,Have_At,URL_Length,URL_Depth,Redirection,https_Domain,TinyURL,Prefix/Suffix,Label
0,icipedudu-my.sharepoint.com,0,0,1,7,0,0,1,1,1
1,ebay.portlibertecontractor.com,0,0,1,3,0,0,0,0,1
2,eliotecae-my.sharepoint.com,0,0,1,5,0,0,1,1,1
3,paypal-uk-webcmd-login.done-login-access-krf41...,0,0,1,4,0,0,0,1,1
4,deliverry-safe.biz,0,0,0,1,0,0,0,1,1
5,cla2020gov.com,0,0,0,1,0,0,0,0,1
6,rakutoen-zpky.cc,0,0,0,0,0,0,0,1,1
7,warrantable-parenth.000webhostapp.com,0,0,0,0,0,0,0,1,1
8,controllolntesasp.com,0,0,0,0,0,0,0,0,1
9,clouddoc-authorize.firebaseapp.com,0,0,0,2,0,0,0,1,1


In [132]:
#converting it to csv
phishing.to_csv('phishing.csv', index= False)

In [122]:
urldata = pd.concat([legitimate, phishing]).reset_index(drop=True)


In [ ]:
urldata.shape

# Storing the data in CSV file
urldata.to_csv('urldata.csv', index=False)

In [ ]:
#unimportant 
import tensorflow 
from tensorflow import keras
keras.__version__